In [1]:
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 1
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None

In [2]:
from src.python.dataframe import Catalogue
from src.python.util import dict_str_hash
from src.python.dataframe import filter
from src.python.util import save_object
from src.python.util import load_object
from src.python.util import save_json
from src.python.util import load_json
import pandas as pd
import numpy as np
from os import path

import matplotlib.pyplot as plt
plt.style.use('ggplot')

VERSION = 1


def newdir(path):
    try:
        os.makedirs(path, exist_ok=True)
    except OSError as error:
        print(f"Error al crear la carpeta: {error}")

# Lectura y limpieza de datos

### Catálogo

Creación de catálogo para re-etiquetar los valores de las bases descargadas


In [3]:
entidad = load_json(
    path.join('data', 'covid', 'history', 'dictionary', 'entidad.json'))

catalogue = Catalogue()
SI_NO = {'1': 'SI', '2': 'NO', '97': 'NA'}
catalogue.add(column='EDAD', name='edad',
              function=lambda x: [int(x) for x in x])
catalogue.add(column='SEXO', name='sexo',
              category={'1': 'MUJER', '2': 'HOMBRE'})
catalogue.add(column='EMBARAZO', name='embarazo',
              category=SI_NO)
catalogue.add(column='NACIONALIDAD', name='nacionalidad',
              category={'1': 'MEXICANA', '2': 'EXTRANGERA'})
catalogue.add(column='MIGRANTE', name='migrante',
              category=SI_NO)
catalogue.add(column='INDIGENA', name='indigena',
              category=SI_NO)
catalogue.add(column='HABLA_LENGUA_INDIG', name='lengua_indigena',
              category=SI_NO)
catalogue.add(column='DIABETES', name='diabetes',
              category=SI_NO)
catalogue.add(column='EPOC', name='epoc',
              category=SI_NO)
catalogue.add(column='ASMA', name='asma',
              category=SI_NO)
catalogue.add(column='INMUSUPR', name='inmunosupresion',
              category=SI_NO)
catalogue.add(column='HIPERTENSION', name='hipertension',
              category=SI_NO)
catalogue.add(column='CARDIOVASCULAR', name='cardiovascular',
              category=SI_NO)
catalogue.add(column='OBESIDAD', name='obesidad',
              category=SI_NO)
catalogue.add(column='RENAL_CRONICA', name='renal_cronica',
              category=SI_NO)
catalogue.add(column='TABAQUISMO', name='tabaquismo',
              category=SI_NO)
catalogue.add(column='OTRA_COM', name='otra_comorbilidad',
              category=SI_NO)
# Variables regresoras
catalogue.add(column='TIPO_PACIENTE', name='tipo',
              category={'1': 'AMBULATORIO', '2': 'HOSPITALIZADO'})
catalogue.add(column='INTUBADO', name='intubado',
              category=SI_NO)
catalogue.add(column='NEUMONIA', name='neumonia',
              category=SI_NO)
catalogue.add(column='UCI', name='uci',
              category=SI_NO)
# Clasificación
catalogue.add(column='OTRO_CASO', name='contacto_covid',
              category=SI_NO)
catalogue.add(column='RESULTADO_LAB', name='laboratorio',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '3': 'PENDIENTE', '4': 'NO VALIDO', '97': 'NA'})
catalogue.add(column='RESULTADO_ANTIGENO', name='antigeno',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '97': 'NA'})
catalogue.add(column='CLASIFICACION_FINAL', name='clasificacion',
              category={'1': 'ASOCIACION', '2': 'DICTAMINACION', '3': 'CONFIRMACION',
                        '4': 'NO VALIDO', '5': 'PENDIENTE', '6': 'SOSPECHOSO', '7': 'NEGATIVO'})
# Fechas
catalogue.add(column='FECHA_INGRESO', name='fecha_ingreso',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_SINTOMAS', name='fecha_sintomas',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_DEF', name='fecha_defuncion',
              default=pd.NaT,
              exception='9999-99-99',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
# Otros
catalogue.add(column='ORIGEN', name='origen',
              category={'1': 'USMER', '2': 'FUERA DE USMER'})
catalogue.add(column='SECTOR', name='sector',
              category={'1': 'CRUZ ROJA', '2': 'DIF', '3': 'ESTATAL', '4': 'IMSS', '5': 'IMSS-BIENESTAR',
                        '6': 'ISSSTE', '7': 'MUNICIPAL', '8': 'PEMEX', '9': 'PRIVADA', '10': 'SEDENA',
                        '11': 'SEMAR', '12': 'SSA', '13': 'UNIVERSITARIO'})
catalogue.add(column='ENTIDAD_UM', name='entidad',
              category=entidad)
catalogue.add(column='ENTIDAD_NAC', name='entidad_nacimiento',
              category=entidad)
catalogue.add(column='ENTIDAD_RES', name='entidad_residencia',
              category=entidad)
catalogue.add(column='MUNICIPIO_RES', name='municipio_residencia',
              function=lambda x: x)
catalogue.add(column='PAIS_NACIONALIDAD', name='pais_nacionalidad',
              function=lambda x: ['NE' if (x == 'SE DESCONOCE' or x == '99')
                                  else x.upper() for x in x])
catalogue.add(column='PAIS_ORIGEN', name='pais_origen',
              function=lambda x: ['NE' if x == 'SE DESCONOCE'
                                  else ('MÉXICO' if x == '97' else x.upper()) for x in x])

### Lectura

Lectura organización y re-etiquetación de bases históricas. La hubicación de los archivos debe de ser `data/covid/history/version-{VERSION}.0`

In [4]:
def read_history(file):
    return pd.read_csv(path.join('data', 'covid', 'history', f'version-{VERSION}.0', file), dtype=str)


# Juntas bases históricas
superdata = read_history('COVID19MEXICO2024.csv')
for subdata in [read_history(f'COVID19MEXICO202{i}.csv') for i in [3, 2, 1, 0]]:
    superdata = pd.concat([superdata,
                           subdata[~subdata.ID_REGISTRO.isin(superdata.ID_REGISTRO)]])

# Guardar contadores iniciales
save_json({col: superdata[col].value_counts().to_dict()
           for col in superdata.columns if col != 'ID_REGISTRO'},
          path.join('data', 'covid', 'history', f'version-{VERSION}.0', 'initial_counts.json'))

superdata = (filter(superdata, catalogue)
             .sort_values('fecha_ingreso').reset_index(drop=True))

# Guardar contadores finales
save_json({col: {str(k): v for k, v in superdata[col].value_counts().to_dict().items()}
           for col in superdata.columns},
          path.join('data', 'covid', 'history', f'version-{VERSION}.0', 'final_counts.json'))

del catalogue, subdata

### Clasificación

Clasificación de bases en "positivos", "negativos" e "indeterminados"

In [5]:
for name, value in {'indeterminados': ['NO VALIDO', 'PENDIENTE', 'SOSPECHOSO'],
                    'positivos': ['ASOCIACION', 'DICTAMINACION', 'CONFIRMACION'],
                    'negativos': ['NEGATIVO']
                    }.items():
    locs = superdata.clasificacion.isin(value)
    newdir(path.join('data', 'covid', 'cleanned'))
    save_object(superdata[locs],
                path.join('data', 'covid', 'cleanned', f'{name}-{VERSION}.0.pkl'))
    superdata = superdata[~locs]
    print(f'{name[0].upper()}{name[1:]}: {sum(locs)}')

del superdata, locs, name, value

Indeterminados: 877943
Positivos: 7714858
Negativos: 11800967


# Creación de datos de entrenamiento

Para red neuronal de clasificación (versión 1)


### Variables de entrenamiento



In [6]:
data = load_object(
    path.join('data', 'covid', 'cleanned', f'positivos-{VERSION}.0.pkl'))

data['indigena'] = data.indigena + '_' + data.lengua_indigena

data['dias'] = [x.days for x
                in data.fecha_ingreso-data.fecha_sintomas]
data['defuncion'] = ~pd.isna(data.fecha_defuncion)
data['grave'] = (data.tipo == 'HOSPITALIZADO') | data['defuncion']

fecha_etapa = np.array([data.fecha_ingreso.min()] +
                       [pd.Timestamp(x) for x in
                        ['2020-09-20', '2021-05-16', '2021-11-21',
                         '2022-04-17', '2022-10-23', '2023-06-25']])

etapa = pd.Series(len(fecha_etapa)-1, index=data.index)
for i in range(len(fecha_etapa)-1):
    etapa[(data.fecha_ingreso >= fecha_etapa[i])
          & (data.fecha_ingreso < fecha_etapa[i+1])] = i
data['etapa'] = etapa

catalogue = Catalogue()
catalogue.add('etapa')
catalogue.add('dias', function=lambda x:
              [max(0, min(16, x)) for x in x])
catalogue.add('edad', function=lambda x:
              [max(0, min(100, x)) for x in x])
catalogue.add(column='sexo', name='mujer', function=lambda x:
              [x == 'MUJER' for x in x])
catalogue.add(column='nacionalidad', name='origen',
              category={'MEXICANA': 'MEXICANO', 'EXTRANGERA': 'EXTRANGERO'})

# catalogue.add('nacionalidad')
catalogue.add('indigena', function=lambda x:
              ['SI' if 'SI' in x else ('NO' if 'NO_NO' == x else 'NE') for x in x])

catalogue.add('migrante', function=lambda x:
              ['NO' if n == 'MEXICANA' else x
                  for x, n in zip(x, data['nacionalidad'])])

for col in ['embarazo', 'diabetes', 'epoc', 'asma', 'inmunosupresion',
            'hipertension', 'cardiovascular', 'obesidad',
            'renal_cronica', 'tabaquismo', 'otra_comorbilidad']:
    catalogue.add(col)
catalogue.add('grave')
catalogue.add('defuncion')
data = filter(data, catalogue)
###########################
data.loc[data.indigena == 'SI', 'origen'] = 'MEXICANO_INDIGENA'
data.loc[data.migrante == 'SI', 'origen'] = 'EXTRANGERO_MIGRANTE'
data.drop(['indigena',	'migrante'], axis=1, inplace=True)
###########################
newdir(path.join('data', 'covid', 'classification', 'dataframe'))
save_object(data,
            path.join('data', 'covid', 'classification', 'dataframe', f'positivos-{VERSION}.0.pkl'))
del data, catalogue, col

### Pre-procesamiento y hashing

In [ ]:
def trainingdata1(data, alpha=5, skip=[], drop=[]):
    skip = skip if isinstance(skip, list) else [skip]
    drop = drop if isinstance(drop, list) else [drop]
    data = data.copy()
    comorb_columns = []
    columns = []
    for col in data.columns:
        if col not in skip:
            counts = data[col].value_counts()
            if 'SI' in counts.index:
                prop = 100 * counts['SI'] / (counts['SI'] + counts['NO'])
                comorb_columns.append(col) if prop < alpha else None
                columns.append(col)
    comorb = pd.Series(0, index=data.index)
    comorb_ne = pd.Series(0, index=data.index)
    for column in columns:
        comorb_ne += (data[column] == 'NE').astype(int)
        if column in comorb_columns:
            comorb += (data[column] == 'SI').astype(int)
        else:
            data[column] = data[column] == 'SI'
    data['comorbilidad'] = comorb
    data['comorbilidad_ne'] = comorb_ne
    grave = data.pop('grave') if 'grave' in data else None
    defuncion = data.pop('defuncion') if 'defuncion' in data else None
    if drop:
        for dp in drop:
            if dp in data.columns:
                data.drop(dp, axis=1, inplace=True)
    data.drop(comorb_columns, axis=1, inplace=True)
    if not (grave is None or 'grave' in drop):
        data['grave'] = grave
    if not (defuncion is None or 'defuncion' in drop):
        data['defuncion'] = defuncion
    return data


def hashing(data):
    grave = data.pop('grave') if 'grave' in data else None
    defuncion = data.pop('defuncion') if 'defuncion' in data else None
    data.index = [dict_str_hash(data.iloc[i]).upper()
                  for i in range(len(data))]
    data.index.name = 'hash'
    if grave is not None:
        data['grave'] = grave.tolist()
    if defuncion is not None:
        data['defuncion'] = defuncion.tolist()

In [ ]:
data = load_object(
    path.join('data', 'covid', 'classification', 'dataframe', f'positivos-{VERSION}.0.pkl'))

# Positivos
positivos = trainingdata1(data, drop='defuncion')
col = positivos.comorbilidad.copy()
col[col >= 2] = 2
positivos.comorbilidad = col
col = positivos.comorbilidad_ne.copy()
col[col >= 1] = 1
positivos.comorbilidad_ne = col
col = positivos.origen.copy()
col[col == 'EXTRANGERO_MIGRANTE'] = 'EXTRANGERO'
positivos.origen = col
col = positivos.etapa.copy()
col[col >= 5] = 5
positivos.etapa = col
hashing(positivos)
save_object(positivos,
            path.join('data', 'covid', 'classification', 'dataframe', f'positivos_hash-{VERSION}.1.pkl'))

# Graves
graves = trainingdata1(data[data.grave],
                       drop=['origen', 'comorbilidad_ne', 'grave'])
col = graves.comorbilidad.copy()
col[col >= 1] = 1
graves.comorbilidad = col
col = graves.etapa.copy()
col[col >= 5] = 5
graves.etapa = col
hashing(graves)
save_object(graves,
            path.join('data', 'covid', 'classification', 'dataframe', f'graves_hash-{VERSION}.1.pkl'))

del data, positivos, graves, col

### Sets de entrenamiento

In [ ]:
def datasets1(data, column, testprop, minsize=10, seed=555):
    # Calcular probabilidad
    muestra = data.index.value_counts()
    muestra = muestra[muestra >= minsize]
    data = data.loc[muestra.index]
    casos = data[data[column]].index.value_counts()
    probabilidad = pd.Series(0.0, index=muestra.index)
    probabilidad.loc[casos.index] = casos / muestra[casos.index]
    # Estandarizar datos
    data = data.drop(column, axis=1).groupby(level=0).head(1)
    data = pd.get_dummies(data.sample(
        len(data), replace=False, random_state=seed))
    data = data[data.columns.sort_values()]
    etiqueta = data.etapa
    for column in data.columns:
        if data[column].dtype.name != 'bool':
            mx = max(data[column])
            if mx > 1.0:
                data[column] = data[column] / mx
    data = data.astype(float)
    data['probabilidad'] = probabilidad.loc[data.index]
    data['muestra'] = muestra.loc[data.index]
    data['etiqueta'] = etiqueta.loc[data.index]
    seed += 1
    # Creación de sets
    test = []
    for label in etiqueta.unique():
        subdata = data[data.etiqueta == label]
        test.append(subdata.sample(round(testprop*len(subdata)),
                                   replace=False,
                                   weights=subdata.muestra,
                                   random_state=seed))
    seed += 1
    test = pd.concat(test, axis=0)
    test = test.sample(len(test), replace=False, random_state=seed)
    train = data.drop(test.index, axis=0)

    testvar = test[['probabilidad', 'muestra', 'etiqueta']]
    trainvar = train[['probabilidad', 'muestra', 'etiqueta']]

    test.drop(['probabilidad', 'muestra', 'etiqueta'], axis=1, inplace=True)
    train.drop(['probabilidad', 'muestra', 'etiqueta'], axis=1, inplace=True)

    return (dict(xtrain=train.values,
                 ytrain=trainvar.probabilidad.to_numpy(),
                 sample=trainvar.muestra.to_numpy(),
                 label=trainvar.etiqueta.to_numpy(),
                 columns=train.columns.to_numpy(),
                 index=train.index.to_numpy()),
            dict(xtest=test.values,
                 ytest=testvar.probabilidad.to_numpy(),
                 sample=testvar.muestra.to_numpy(),
                 label=testvar.etiqueta.to_numpy(),
                 columns=test.columns.to_numpy(),
                 index=test.index.to_numpy()))


newdir(path.join('data', 'covid', 'classification', 'datasets'))

train, test = datasets1(data=load_object(
    path.join('data', 'covid', 'classification', 'dataframe', f'graves_hash-{VERSION}.1.pkl')),
    column='defuncion', testprop=0.15)
save_object(train,
            path.join('data', 'covid', 'classification', 'datasets', f'graves_train-{VERSION}.1.pkl'))
save_object(test,
            path.join('data', 'covid', 'classification', 'datasets', f'graves_test-{VERSION}.1.pkl'))
train, test = positivos = datasets1(data=load_object(
    path.join('data', 'covid', 'classification', 'dataframe', f'positivos_hash-{VERSION}.1.pkl')),
    column='grave', testprop=0.2)
save_object(train,
            path.join('data', 'covid', 'classification', 'datasets', f'positivos_train-{VERSION}.1.pkl'))
save_object(test,
            path.join('data', 'covid', 'classification', 'datasets', f'positivos_test-{VERSION}.1.pkl'))

del train, test